# Overview

This is an updated form that takes the place of the original NLSS notebook. However, it uses the more complete CSV files instead of the three smaller CSVs used in the original. In this Notebook, I organize data from a text document of dockets. The dockets include the date of the stream, who was on the show, and what games were played. I split up the data into categories and make lists out of the participants and games. I also take data on stream statistics from a CSV file. The data from the dockets and the stream stats are put together into one dataframe.

In [1]:
# Every returned Out[] is displayed, not just the last one. 
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
import nltk  
import pandas as pd
import numpy as np

In [3]:
#Open up the docket and take a look
with open(r'PublicData\NLSS_Dockets.txt') as f:
    file = f.read()
shows = file.split('\n\n') #split into every show
shows[:5]

['(August 24, 2017) Nick View (NL, RLS, CS, rob)\nPasspartout, Party Panic, Pinturillo',
 '(August 23, 2017) Nick View (NL, RLS, rob w/ Baer, LGW, HCJ)\nAbsolver, Golf It, Quiplash',
 '(August 21, 2017) Nick View (NL, RLS, JS, rob)\nFire Pro Wrestling World, Ultimate Chicken Horse, Blood Party',
 '(August 17, 2017) Nick View (NL w/ Sin, RLS, LGW, HCJ, Baer)\nGeoguessr, Golf It, Quiplash',
 '(August 16, 2017) Nick View (NL, RLS w/ rob, Baer, LGW, Dan)\nNidhogg 2, Speedrunners, Pinturillo']

In [4]:
#Take out unneeded text
index = 0
for s in shows:
    shows[index] = s.replace(' Nick View', '')
    index+=1
shows[-10:]

['(March 27, 2013) (NL, RLS, JS w/ Ohm)\nDark Souls Invasions, Trivia, Arma III, Ask me anything on Twitter',
 '(March 18, 2013) (NL, RLS, JS)\nDark Souls Invasions, Trivia, Rollercoaster Tycoon, More Dark Souls Invasions, Ask me anything on Twitter',
 '(March 14, 2013) (NL, RLS, JS w/ MALF)\nDark Souls Invasions, Trivia, Worms Revolution, Ask me anything on Twitter',
 '(March 13, 2013) (NL, RLS, JS w/ Ohm)\nDark Souls Invasions, Trivia, The Showdown Effect',
 '(March 11, 2013) (NL, RLS, JS w/ Ohm)\nDark Souls Invasions, Trivia, More Dark Souls Invasions, Arma III, Ask me anything on Twitter',
 '(March 6, 2013) (NL, RLS, JS)\nDark Souls invasions, Trivia, Tomb Raider, Ask me anything on Twitter',
 "(March 4, 2013) (NL, RLS, JS)\nDelver's Drop with Ryan Baker and Ryan Burrell, Dark Souls, Trivia, More Dark Souls, Ask me anything on Twitter",
 '(February 28, 2013) (NL, RLS, JS)\nDark Souls, Trivia, Trials Evolution, Ask me anything on Twitter',
 '(February 27, 2013) (NL, Kate)\nDark Soul

In [5]:
#Create a list of games
games = []
for s in shows:
    g = s.split('\n') #Text files has games on second line
    games.append(g[1])
games

['Passpartout, Party Panic, Pinturillo',
 'Absolver, Golf It, Quiplash',
 'Fire Pro Wrestling World, Ultimate Chicken Horse, Blood Party',
 'Geoguessr, Golf It, Quiplash',
 'Nidhogg 2, Speedrunners, Pinturillo',
 'Hitman, Ben & Edd Blood Party, London 2012',
 'Ultimate Chicken Horse, Tower Unite, Who Wants To Be A Millionaire, Super Hexagon',
 'Passpartout, Witch It, Quiplash',
 'Afterbirth+, Golf with your Friends, Guesspionage',
 'Tricky Towers, Sonic and All Stars Racing Transformed, London 2012',
 'Ben and Ed - Blood Party, Ball 3D: Soccer Online, Quiplash',
 'Fortnite, Golf It, Gang Beasts',
 'Cannon Brawl, Goldeneye: Source, Ball 3D: Soccer Online',
 'Hitman, Ben & Ed - Blood Party, Tower Unite',
 'Ben and Ed - Blood Party, Golf with Your Friends, Pinturillo',
 "Ben and Edd's Blood Party, London 2012, Quiplash",
 'Passpartout, Ultimate Chicken Horse, Tower Unite',
 'The End Is Nigh, Passpartout, Quiplash',
 'Hitman, (continued), Tower Unite, Guesspionage',
 'Passpartout: The Star

In [6]:
#Create a list of crew members and show dates
date_crew = []
for s in shows:
    dc = s.split('\n')[0]
    date_crew.append(dc)
print(date_crew)

['(August 24, 2017) (NL, RLS, CS, rob)', '(August 23, 2017) (NL, RLS, rob w/ Baer, LGW, HCJ)', '(August 21, 2017) (NL, RLS, JS, rob)', '(August 17, 2017) (NL w/ Sin, RLS, LGW, HCJ, Baer)', '(August 16, 2017) (NL, RLS w/ rob, Baer, LGW, Dan)', '(August 14, 2017) (NL, JS, MALF, LGW w/ Baer, HCJ)', '(August 10, 2017) (NL, RLS, rob, LGW)', '(August 7, 2017) (NL, RLS, JS, rob w/ Baer)', '(August 3, 2017) (NL, RLS, CS w/ rob, MALF)', '(August 2, 2017) (NL, RLS, LGW w/ Baer, Kory)', '(July 31, 2017) (NL, RLS, JS, rob w/ Sin, Baer, TB)', '(July 27, 2017) (NL, RLS, CS w/ MALF)', '(July 26, 2017) (NL, RLS w/ LGW, Sin, Baer, Dan)', '(July 24, 2017) (NL, RLS, JS w/ LGW)', '(July 20, 2017) (NL, RLS, CS, LGW w/ Baer)', '(July 19, 2017) (NL, RLS, LGW w/ MALF, rob, Baer)', '(July 13, 2017) (NL, RLS, rob w/ LGW, Baer)', '(July 12, 2017) (NL, RLS, rob w/ Baer)', '(July 10, 2017) part 1, part 2 (NL, RLS, JS w/ rob, Sin)', '(July 6, 2017) (NL, RLS, CS, rob w/ Baer)', '(July 5, 2017) (NL, RLS, rob w/ LGW, 

In [7]:
#Split into list for crew and list for dates
import re
date = []
crew = []
for entry in date_crew:
    foo = re.search(r'\((.*)\)', entry).group(1)
    d = foo.split(r')')[0]
    date.append(d)
    c = foo.split(r'(')[-1]
    crew.append(c)

In [8]:
date_df = pd.DataFrame(date, columns = ["Date"])
date_df.head()

,Date
0,"August 24, 2017"
1,"August 23, 2017"
2,"August 21, 2017"
3,"August 17, 2017"
4,"August 16, 2017"


In [9]:
games_df = pd.DataFrame(games, columns = ["Docket"])
games_df.head()

,Docket
0,"Passpartout, Party Panic, Pinturillo"
1,"Absolver, Golf It, Quiplash"
2,"Fire Pro Wrestling World, Ultimate Chicken Hor..."
3,"Geoguessr, Golf It, Quiplash"
4,"Nidhogg 2, Speedrunners, Pinturillo"


In [10]:
crew_df = pd.DataFrame(crew, columns = ["Crew"])
crew_df.head()

,Crew
0,"NL, RLS, CS, rob"
1,"NL, RLS, rob w/ Baer, LGW, HCJ"
2,"NL, RLS, JS, rob"
3,"NL w/ Sin, RLS, LGW, HCJ, Baer"
4,"NL, RLS w/ rob, Baer, LGW, Dan"


In [11]:
#Put these data frames together
nlss_df = pd.DataFrame()
nlss_df['Date'] = date_df['Date']
nlss_df['Crew'] = crew_df['Crew']
nlss_df['Docket'] = games_df['Docket']
nlss_df.head()

,Date,Crew,Docket
0,"August 24, 2017","NL, RLS, CS, rob","Passpartout, Party Panic, Pinturillo"
1,"August 23, 2017","NL, RLS, rob w/ Baer, LGW, HCJ","Absolver, Golf It, Quiplash"
2,"August 21, 2017","NL, RLS, JS, rob","Fire Pro Wrestling World, Ultimate Chicken Hor..."
3,"August 17, 2017","NL w/ Sin, RLS, LGW, HCJ, Baer","Geoguessr, Golf It, Quiplash"
4,"August 16, 2017","NL, RLS w/ rob, Baer, LGW, Dan","Nidhogg 2, Speedrunners, Pinturillo"


In [12]:
improved = []
#For each docket
for d in nlss_df['Docket']:
    #Split docket into list of games
    d = d.split(r',')
    #For each game
    for g in d:
        #If game matches string to remove
        if g == r" (continued)" or g == r" (Continued)":
            #Remove game
            d.remove(g)
    improved.append(d)
nlss_df['Docket'] = improved

In [13]:
improved = []
#For each cast of crew
for e in nlss_df['Crew']:
    #Split cast into list of members
    e = e.split(r',')
    #For each member
    for m in e:
        #If member contains a /w
        if r'w/' in m:
            both = m.split(r'w/')
            e.remove(m)
            e.extend(both)
    improved.append(e)
improved[:20]

[['NL', ' RLS', ' CS', ' rob'],
 ['NL', ' RLS', ' LGW', ' HCJ', ' rob ', ' Baer'],
 ['NL', ' RLS', ' JS', ' rob'],
 [' RLS', ' LGW', ' HCJ', ' Baer', 'NL ', ' Sin'],
 ['NL', ' Baer', ' LGW', ' Dan', ' RLS ', ' rob'],
 ['NL', ' JS', ' MALF', ' HCJ', ' LGW ', ' Baer'],
 ['NL', ' RLS', ' rob', ' LGW'],
 ['NL', ' RLS', ' JS', ' rob ', ' Baer'],
 ['NL', ' RLS', ' MALF', ' CS ', ' rob'],
 ['NL', ' RLS', ' Kory', ' LGW ', ' Baer'],
 ['NL', ' RLS', ' JS', ' Baer', ' TB', ' rob ', ' Sin'],
 ['NL', ' RLS', ' CS ', ' MALF'],
 ['NL', ' Sin', ' Baer', ' Dan', ' RLS ', ' LGW'],
 ['NL', ' RLS', ' JS ', ' LGW'],
 ['NL', ' RLS', ' CS', ' LGW ', ' Baer'],
 ['NL', ' RLS', ' rob', ' Baer', ' LGW ', ' MALF'],
 ['NL', ' RLS', ' Baer', ' rob ', ' LGW'],
 ['NL', ' RLS', ' rob ', ' Baer'],
 ['NL', ' RLS', ' Sin', ' JS ', ' rob'],
 ['NL', ' RLS', ' CS', ' rob ', ' Baer']]

In [14]:
#Clean up members list
fullstripped = []
for entry in improved:
    stripped = []
    for member in entry:
        member = member.strip(' ')
        stripped.append(member)
    fullstripped.append(stripped)
fullstripped[:10]

[['NL', 'RLS', 'CS', 'rob'],
 ['NL', 'RLS', 'LGW', 'HCJ', 'rob', 'Baer'],
 ['NL', 'RLS', 'JS', 'rob'],
 ['RLS', 'LGW', 'HCJ', 'Baer', 'NL', 'Sin'],
 ['NL', 'Baer', 'LGW', 'Dan', 'RLS', 'rob'],
 ['NL', 'JS', 'MALF', 'HCJ', 'LGW', 'Baer'],
 ['NL', 'RLS', 'rob', 'LGW'],
 ['NL', 'RLS', 'JS', 'rob', 'Baer'],
 ['NL', 'RLS', 'MALF', 'CS', 'rob'],
 ['NL', 'RLS', 'Kory', 'LGW', 'Baer']]

In [15]:
#A list of all unique members
names = []
for entry in fullstripped:
    for user in entry:
        if user not in names:
            names.append(user)
print(names)

['NL', 'RLS', 'CS', 'rob', 'LGW', 'HCJ', 'Baer', 'JS', 'Sin', 'Dan', 'MALF', 'Kory', 'TB', 'Kate', 'Blueman', 'BaerBaer', 'Mathas', 'Crendor', 'BRex', 'GhostBill', 'alpacapatrol', 'dan', '', 'Brex', 'Fox', 'Arumba', 'baer', 'cobaltstreak', 'fox', 'Mag', 'NLS', 'JS!', 'RLS in person', 'Kate in person', 'Mike Bithell', 'RPG', 'Ohm', 'Pixel', 'Green']


In [16]:
#Clean up the list so that it follows a consistent format
foo = "Northernlion, RockLeeSmile, CobaltStreak, AlpacaPatrol, LastGreyWolf, HCJustin, BaerTaffy, JSmithOTI, Sinvicta, DanGheesling, MALF, FlackBlag, TotalBiscuit, LovelyMomo, Blueman, BaerTaffy, MathasGames, Crendor, BananasaurusRex, NOTREAL, AlpacaPatrol, DanGheesling, NOTREAL, BananasaurusRex, MALF, Arumba, BaerTaffy, CobaltStreak, MALF, Magresta, Northernlion, JSmithOTI, RockLeeSmile, LovelyMomo, MikeBithell, RedPandaGamer, OhmWrecker, PrescriptionPixel, Green9090"
translated = foo.split(", ")
translated

['Northernlion',
 'RockLeeSmile',
 'CobaltStreak',
 'AlpacaPatrol',
 'LastGreyWolf',
 'HCJustin',
 'BaerTaffy',
 'JSmithOTI',
 'Sinvicta',
 'DanGheesling',
 'MALF',
 'FlackBlag',
 'TotalBiscuit',
 'LovelyMomo',
 'Blueman',
 'BaerTaffy',
 'MathasGames',
 'Crendor',
 'BananasaurusRex',
 'NOTREAL',
 'AlpacaPatrol',
 'DanGheesling',
 'NOTREAL',
 'BananasaurusRex',
 'MALF',
 'Arumba',
 'BaerTaffy',
 'CobaltStreak',
 'MALF',
 'Magresta',
 'Northernlion',
 'JSmithOTI',
 'RockLeeSmile',
 'LovelyMomo',
 'MikeBithell',
 'RedPandaGamer',
 'OhmWrecker',
 'PrescriptionPixel',
 'Green9090']

In [17]:
guests = []
for cast in fullstripped:
    guests.append([translated[names.index(user)] for user in cast])

In [18]:
#Replace first names with second names
guests[0]

['Northernlion', 'RockLeeSmile', 'CobaltStreak', 'AlpacaPatrol']

In [19]:
nlss_df['Crew'] = guests
nlss_df.head()
nlss_df.tail()

,Date,Crew,Docket
0,"August 24, 2017","[Northernlion, RockLeeSmile, CobaltStreak, Alp...","[Passpartout, Party Panic, Pinturillo]"
1,"August 23, 2017","[Northernlion, RockLeeSmile, LastGreyWolf, HCJ...","[Absolver, Golf It, Quiplash]"
2,"August 21, 2017","[Northernlion, RockLeeSmile, JSmithOTI, Alpaca...","[Fire Pro Wrestling World, Ultimate Chicken H..."
3,"August 17, 2017","[RockLeeSmile, LastGreyWolf, HCJustin, BaerTaf...","[Geoguessr, Golf It, Quiplash]"
4,"August 16, 2017","[Northernlion, BaerTaffy, LastGreyWolf, DanGhe...","[Nidhogg 2, Speedrunners, Pinturillo]"


,Date,Crew,Docket
583,"March 6, 2013","[Northernlion, RockLeeSmile, JSmithOTI]","[Dark Souls invasions, Trivia, Tomb Raider, ..."
584,"March 4, 2013","[Northernlion, RockLeeSmile, JSmithOTI]",[Delver's Drop with Ryan Baker and Ryan Burrel...
585,"February 28, 2013","[Northernlion, RockLeeSmile, JSmithOTI]","[Dark Souls, Trivia, Trials Evolution, Ask ..."
586,"February 27, 2013","[Northernlion, LovelyMomo]","[Dark Souls, Trivia, More Dark Souls, Ask m..."
587,"February 25, 2013",[Northernlion],"[Runner 2, Super House of Dead Ninjas, Trivi..."


## FULL CSV

Now import the stream stats CSV

In [20]:
import os
import glob
print(os.getcwd())

stream_df = pd.read_csv(r'PublicData\Northernlion_Full.csv',index_col=None, header=0)
stream_df.drop('GamesPlayed',axis=1, inplace=True)
stream_df[150:160]

C:\Users\Sonof\Documents\Data_Science\NLSS_Project


FileNotFoundError: File b'data\\FullCSV\\Northernlion_Full.csv' does not exist

In [ ]:
#Remove extra line we don't need
stream_df.drop(154, inplace=True)
stream_df[150:160]

In [ ]:
datetime = [date.split(' ')for date in stream_df['StartTime']]
datetime

In [ ]:
months = {'01':'January', '02':'February', '03':'March', '04':'April', '05':'May', '06':'June', '07':'July', '08':'August', '09':'September', '10':'October', '11':'November', '12':'December'}

In [ ]:
#Make the dates in the CSV match the format of the docket
formatted = []
for stream in datetime:
    ymd = stream[0].split('-')
    entry = months.get(ymd[1])+" "+ymd[2]+", "+ymd[0]
    formatted.append(entry)
formatted

In [ ]:
stream_df['Date'] = formatted
stream_df.head()
len(stream_df)

In [ ]:
combined = pd.merge(nlss_df, stream_df)
combined.head()
combined.tail()
len(combined)

Almost 200 lines that overlap. These are the videos we will get the comments for.